# Resume-JD Matching and ATS Scoring Engine

This notebook evaluates how well a resume aligns with a job description.
- **Enhanced skill extraction** with comprehensive tech stack coverage
- **Context-aware parsing** for better experience and project detection
- **Visual heatmap** showing skill gaps
- **Accurate scoring** reflecting true readiness for the role

In [ ]:
# Install required packages
!pip install -q PyPDF2 python-docx scikit-learn numpy matplotlib

In [ ]:
# Configuration - DO NOT MODIFY WEIGHTS
SCORING_WEIGHTS = {
    'skill_match': 0.45,
    'experience_fit': 0.25,
    'project_relevance': 0.20,
    'bonus_signals': 0.10
}

# Comprehensive skill database with variations
SKILL_DATABASE = {
    # Programming Languages
    'python': ['python', 'py', 'python3', 'django', 'flask', 'fastapi'],
    'java': ['java', 'j2ee', 'spring', 'springboot', 'hibernate'],
    'javascript': ['javascript', 'js', 'node', 'nodejs', 'node.js', 'typescript', 'ts'],
    'c++': ['c++', 'cpp', 'cplusplus'],
    'c#': ['c#', 'csharp', '.net', 'dotnet', 'asp.net'],
    'go': ['go', 'golang'],
    'rust': ['rust'],
    'php': ['php', 'laravel', 'symfony'],
    'ruby': ['ruby', 'rails', 'ruby on rails'],
    'swift': ['swift', 'ios'],
    'kotlin': ['kotlin', 'android'],
    
    # Web Technologies
    'html': ['html', 'html5'],
    'css': ['css', 'css3', 'sass', 'scss', 'less'],
    'react': ['react', 'reactjs', 'react.js', 'react native', 'redux', 'next.js', 'nextjs'],
    'angular': ['angular', 'angularjs'],
    'vue': ['vue', 'vuejs', 'vue.js', 'nuxt'],
    
    # Databases
    'sql': ['sql', 'mysql', 'postgresql', 'postgres', 'sqlite', 'mariadb'],
    'mongodb': ['mongodb', 'mongo', 'nosql'],
    'redis': ['redis', 'cache'],
    'elasticsearch': ['elasticsearch', 'elastic'],
    
    # Data Science & ML
    'pandas': ['pandas', 'dataframe'],
    'numpy': ['numpy', 'numerical'],
    'scikit-learn': ['scikit-learn', 'sklearn', 'machine learning', 'ml'],
    'tensorflow': ['tensorflow', 'tf', 'keras'],
    'pytorch': ['pytorch', 'torch'],
    'opencv': ['opencv', 'cv2', 'computer vision'],
    
    # DevOps & Tools
    'docker': ['docker', 'container', 'containerization'],
    'kubernetes': ['kubernetes', 'k8s', 'orchestration'],
    'git': ['git', 'github', 'gitlab', 'version control'],
    'jenkins': ['jenkins', 'ci/cd', 'continuous integration'],
    'aws': ['aws', 'amazon web services', 'ec2', 's3', 'lambda'],
    'azure': ['azure', 'microsoft azure'],
    'gcp': ['gcp', 'google cloud', 'google cloud platform'],
    
    # Other Technologies
    'rest': ['rest', 'restful', 'api', 'rest api'],
    'graphql': ['graphql', 'apollo'],
    'agile': ['agile', 'scrum', 'kanban', 'jira'],
    'testing': ['testing', 'unit test', 'pytest', 'jest', 'junit'],
}

print("✅ Configuration loaded")

In [ ]:
import PyPDF2
import re
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

def extract_text_from_pdf(file_path):
    """Extract text from PDF file"""
    text = ""
    try:
        with open(file_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        print(f"Error reading PDF: {e}")
    return text

def extract_text_from_docx(file_path):
    """Extract text from DOCX file"""
    try:
        import docx
        doc = docx.Document(file_path)
        return "\n".join([p.text for p in doc.paragraphs])
    except Exception as e:
        print(f"Error reading DOCX: {e}")
        return ""

def extract_text(file_path):
    """Universal text extractor"""
    path = Path(file_path)
    
    if path.suffix.lower() == '.pdf':
        return extract_text_from_pdf(file_path)
    elif path.suffix.lower() == '.docx':
        return extract_text_from_docx(file_path)
    elif path.suffix.lower() == '.txt':
        return open(file_path, 'r', encoding='utf-8').read()
    else:
        return ""

print("✅ Text extraction functions loaded")

In [ ]:
class EnhancedResumeParser:
    """Enhanced parser with comprehensive skill detection"""
    
    def __init__(self, skill_db):
        self.skill_db = skill_db
    
    def parse(self, text):
        text_lower = text.lower()
        
        return {
            "skills": self._extract_skills(text_lower),
            "experience_years": self._extract_years(text_lower),
            "projects": self._extract_projects(text),
            "certifications": self._extract_certifications(text),
            "education": self._extract_education(text_lower)
        }
    
    def _extract_skills(self, text):
        """Extract skills using comprehensive database"""
        found_skills = set()
        
        for skill_name, variations in self.skill_db.items():
            for variation in variations:
                # Use word boundaries for accurate matching
                pattern = r'\b' + re.escape(variation) + r'\b'
                if re.search(pattern, text):
                    found_skills.add(skill_name)
                    break
        
        return list(found_skills)
    
    def _extract_years(self, text):
        """Extract years of experience with multiple patterns"""
        patterns = [
            r'(\d+\.?\d*)\s*(?:\+)?\s*years?\s+(?:of\s+)?experience',
            r'experience\s*:?\s*(\d+\.?\d*)\s*(?:\+)?\s*years?',
            r'(\d+\.?\d*)\s*(?:\+)?\s*yrs?\s+(?:of\s+)?experience',
        ]
        
        for pattern in patterns:
            match = re.search(pattern, text)
            if match:
                return float(match.group(1))
        
        return 0.0
    
    def _extract_projects(self, text):
        """Extract project descriptions"""
        projects = []
        lines = text.split('\n')
        
        # Look for project sections
        in_project_section = False
        for line in lines:
            line_lower = line.lower()
            
            # Detect project section headers
            if re.search(r'\b(project|work)\b', line_lower) and len(line) < 50:
                in_project_section = True
                continue
            
            # Collect substantial project lines
            if in_project_section and len(line.strip()) > 40:
                projects.append(line.strip())
                if len(projects) >= 5:
                    break
        
        return projects
    
    def _extract_certifications(self, text):
        """Extract certifications"""
        cert_keywords = [
            'certified', 'certification', 'certificate', 'credential',
            'aws certified', 'google cloud', 'microsoft certified',
            'coursera', 'udemy', 'professional certificate'
        ]
        
        certifications = []
        for line in text.split('\n'):
            line_lower = line.lower()
            if any(keyword in line_lower for keyword in cert_keywords):
                certifications.append(line.strip())
        
        return certifications
    
    def _extract_education(self, text):
        """Extract education level"""
        if re.search(r'\b(phd|ph\.d|doctorate)\b', text):
            return 'phd'
        elif re.search(r'\b(master|m\.s|m\.tech|mba)\b', text):
            return 'masters'
        elif re.search(r'\b(bachelor|b\.s|b\.tech|b\.e)\b', text):
            return 'bachelors'
        return 'other'

class EnhancedJDParser:
    """Enhanced JD parser with better requirement extraction"""
    
    def __init__(self, skill_db):
        self.skill_db = skill_db
    
    def parse(self, text):
        text_lower = text.lower()
        
        required, preferred = self._categorize_skills(text_lower)
        
        return {
            "required_skills": required,
            "preferred_skills": preferred,
            "min_experience": self._extract_years(text_lower),
            "role_level": self._extract_level(text_lower)
        }
    
    def _categorize_skills(self, text):
        """Separate required vs preferred skills"""
        required = set()
        preferred = set()
        
        # Look for skills in required section
        req_section = re.search(r'(?:required|must have|essential)[\s\S]*?(?:preferred|nice|optional|$)', text)
        pref_section = re.search(r'(?:preferred|nice to have|optional|bonus)[\s\S]*?(?:$)', text)
        
        for skill_name, variations in self.skill_db.items():
            for variation in variations:
                pattern = r'\b' + re.escape(variation) + r'\b'
                
                # Check in required section
                if req_section and re.search(pattern, req_section.group()):
                    required.add(skill_name)
                    break
                
                # Check in preferred section
                if pref_section and re.search(pattern, pref_section.group()):
                    preferred.add(skill_name)
                    break
                
                # If no sections found, check in full text
                if not req_section and not pref_section:
                    if re.search(pattern, text):
                        required.add(skill_name)
                        break
        
        return list(required), list(preferred)
    
    def _extract_years(self, text):
        """Extract minimum years of experience"""
        patterns = [
            r'(\d+\.?\d*)\s*(?:\+)?\s*years?\s+(?:of\s+)?experience',
            r'minimum\s+(\d+\.?\d*)\s*(?:\+)?\s*years?',
        ]
        
        for pattern in patterns:
            match = re.search(pattern, text)
            if match:
                return float(match.group(1))
        
        return 0.0
    
    def _extract_level(self, text):
        """Determine role level"""
        if re.search(r'\b(intern|internship)\b', text):
            return "intern"
        elif re.search(r'\b(entry|junior|fresher|graduate)\b', text):
            return "junior"
        elif re.search(r'\b(mid|intermediate|senior)\b', text):
            return "mid"
        elif re.search(r'\b(lead|principal|architect|staff)\b', text):
            return "senior"
        return "mid"

print("✅ Enhanced parsers loaded")

In [ ]:
class ScoringEngine:
    """Scoring engine with fixed weights"""
    
    def __init__(self, weights):
        self.weights = weights
    
    def calculate_skill_score(self, matched, required):
        """Calculate skill match score"""
        if not required:
            return 90.0  # If no requirements, assume good match
        
        match_ratio = len(matched) / len(required)
        
        # Progressive scoring
        if match_ratio >= 0.9:
            return 95.0
        elif match_ratio >= 0.75:
            return 85.0
        elif match_ratio >= 0.6:
            return 75.0
        elif match_ratio >= 0.4:
            return 60.0
        elif match_ratio >= 0.2:
            return 40.0
        else:
            return 20.0
    
    def calculate_experience_score(self, resume_years, required_years):
        """Calculate experience fit score"""
        if required_years == 0:
            return 80.0
        
        if resume_years >= required_years:
            # Bonus for exceeding requirements
            excess = min((resume_years - required_years) / required_years, 0.5)
            return min(100.0, 90.0 + (excess * 20))
        else:
            # Penalty for insufficient experience
            deficit = (required_years - resume_years) / required_years
            return max(30.0, 90.0 - (deficit * 60))
    
    def calculate_project_score(self, projects):
        """Calculate project relevance score"""
        project_count = len(projects)
        
        if project_count >= 4:
            return 90.0
        elif project_count >= 3:
            return 80.0
        elif project_count >= 2:
            return 65.0
        elif project_count >= 1:
            return 50.0
        else:
            return 25.0
    
    def calculate_bonus_score(self, certifications, education, preferred_match_count):
        """Calculate bonus signals score"""
        score = 40.0  # Base score
        
        # Certifications bonus
        if len(certifications) >= 3:
            score += 25.0
        elif len(certifications) >= 1:
            score += 15.0
        
        # Education bonus
        if education == 'phd':
            score += 20.0
        elif education == 'masters':
            score += 15.0
        elif education == 'bachelors':
            score += 10.0
        
        # Preferred skills bonus
        if preferred_match_count >= 3:
            score += 15.0
        elif preferred_match_count >= 1:
            score += 10.0
        
        return min(100.0, score)
    
    def final_score(self, scores):
        """Calculate weighted final score"""
        total = sum(scores[k] * self.weights[k] for k in scores)
        total = round(total, 2)
        
        # Determine readiness level
        if total >= 85:
            level = "Highly Prepared"
            msg = "Excellent alignment with this role"
        elif total >= 70:
            level = "Prepared"
            msg = "Strong alignment, ready to apply"
        elif total >= 55:
            level = "Developing Readiness"
            msg = "Good foundation, some skill gaps to address"
        elif total >= 40:
            level = "Preparation Stage"
            msg = "Basic alignment, significant improvements needed"
        else:
            level = "Early Stage"
            msg = "Role requirements significantly exceed current profile"
        
        return total, level, msg

print("✅ Scoring engine loaded")

In [ ]:
class ATSMatcher:
    """Main ATS matching engine"""
    
    def __init__(self, skill_db, weights):
        self.resume_parser = EnhancedResumeParser(skill_db)
        self.jd_parser = EnhancedJDParser(skill_db)
        self.scorer = ScoringEngine(weights)
    
    def match(self, resume_text, jd_text):
        """Perform comprehensive resume-JD matching"""
        
        # Parse documents
        resume = self.resume_parser.parse(resume_text)
        jd = self.jd_parser.parse(jd_text)
        
        # Calculate skill matches
        matched_skills = list(set(resume["skills"]) & set(jd["required_skills"]))
        missing_skills = list(set(jd["required_skills"]) - set(resume["skills"]))
        preferred_matched = list(set(resume["skills"]) & set(jd["preferred_skills"]))
        preferred_missing = list(set(jd["preferred_skills"]) - set(resume["skills"]))
        
        # Calculate individual scores
        skill_score = self.scorer.calculate_skill_score(matched_skills, jd["required_skills"])
        exp_score = self.scorer.calculate_experience_score(resume["experience_years"], jd["min_experience"])
        project_score = self.scorer.calculate_project_score(resume["projects"])
        bonus_score = self.scorer.calculate_bonus_score(
            resume["certifications"],
            resume["education"],
            len(preferred_matched)
        )
        
        scores = {
            "skill_match": skill_score,
            "experience_fit": exp_score,
            "project_relevance": project_score,
            "bonus_signals": bonus_score
        }
        
        final, level, msg = self.scorer.final_score(scores)
        
        return {
            "final_score": final,
            "readiness_level": level,
            "summary": msg,
            "strengths": matched_skills,
            "missing_required": missing_skills,
            "preferred_matched": preferred_matched,
            "preferred_missing": preferred_missing,
            "scores": scores,
            "resume_details": {
                "experience_years": resume["experience_years"],
                "project_count": len(resume["projects"]),
                "certification_count": len(resume["certifications"]),
                "education": resume["education"]
            },
            "jd_details": {
                "required_experience": jd["min_experience"],
                "role_level": jd["role_level"]
            }
        }
    
    def print_detailed_report(self, report):
        """Print comprehensive report"""
        print("=" * 70)
        print("               ATS SCORE REPORT")
        print("=" * 70)
        print(f"\n🎯 FINAL SCORE: {report['final_score']}/100")
        print(f"📊 READINESS: {report['readiness_level']}")
        print(f"💡 {report['summary']}")
        
        print("\n" + "-" * 70)
        print("SCORE BREAKDOWN:")
        print("-" * 70)
        for key, value in report['scores'].items():
            weight = SCORING_WEIGHTS[key] * 100
            contribution = value * SCORING_WEIGHTS[key]
            print(f"  {key.replace('_', ' ').title():25} {value:5.1f}% (Weight: {weight:2.0f}%) → {contribution:5.2f}")
        
        print("\n" + "-" * 70)
        print("CANDIDATE PROFILE:")
        print("-" * 70)
        details = report['resume_details']
        print(f"  Experience: {details['experience_years']:.1f} years")
        print(f"  Projects: {details['project_count']}")
        print(f"  Certifications: {details['certification_count']}")
        print(f"  Education: {details['education'].title()}")
        
        print("\n" + "-" * 70)
        print("✅ MATCHING SKILLS ({}):\".format(len(report['strengths'])))
        print("-" * 70)
        if report['strengths']:
            for skill in sorted(report['strengths']):
                print(f"  • {skill.upper()}")
        else:
            print("  No matching required skills found")
        
        print("\n" + "-" * 70)
        print(f"❌ MISSING REQUIRED SKILLS ({len(report['missing_required'])}):")
        print("-" * 70)
        if report['missing_required']:
            for skill in sorted(report['missing_required']):
                print(f"  • {skill.upper()}")
        else:
            print("  All required skills present!")
        
        print("\n" + "-" * 70)
        print(f"⭐ BONUS: PREFERRED SKILLS ({len(report['preferred_matched'])}):")
        print("-" * 70)
        if report['preferred_matched']:
            for skill in sorted(report['preferred_matched']):
                print(f"  • {skill.upper()}")
        else:
            print("  None matched")
        
        if report['preferred_missing']:
            print(f"\n💡 Growth Opportunities ({len(report['preferred_missing'])}):")
            for skill in sorted(report['preferred_missing'])[:5]:
                print(f"  • {skill.upper()}")
        
        print("\n" + "=" * 70)

print("✅ ATS Matcher loaded")

In [ ]:
def plot_enhanced_heatmap(report):
    """
    Create professional skill gap heatmap with detailed breakdown
    """
    skills = []
    scores = []
    colors = []
    labels = []
    
    # Matched skills (green)
    for skill in sorted(report['strengths']):
        skills.append(skill.upper())
        scores.append(95)
        colors.append("#2ecc71")
        labels.append("Required ✓")
    
    # Preferred matched (blue)
    for skill in sorted(report['preferred_matched'])[:5]:
        skills.append(skill.upper())
        scores.append(75)
        colors.append("#3498db")
        labels.append("Preferred ✓")
    
    # Preferred missing (yellow)
    for skill in sorted(report['preferred_missing'])[:5]:
        skills.append(skill.upper())
        scores.append(50)
        colors.append("#f39c12")
        labels.append("Preferred ✗")
    
    # Missing required (red)
    for skill in sorted(report['missing_required'])[:8]:
        skills.append(skill.upper())
        scores.append(20)
        colors.append("#e74c3c")
        labels.append("Required ✗")
    
    if not skills:
        print("⚠️  No skills to visualize")
        return
    
    # Create figure
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), 
                                     gridspec_kw={'height_ratios': [3, 1]})
    
    # Main heatmap
    bars = ax1.bar(skills, scores, color=colors, edgecolor="black", linewidth=0.8, alpha=0.85)
    
    # Add percentage labels on bars
    for bar, score in zip(bars, scores):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 1,
                f'{score}%',
                ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    ax1.set_ylim(0, 105)
    ax1.set_ylabel('Readiness Level (%)', fontsize=12, fontweight='bold')
    ax1.set_title(f'ATS Score: {report["final_score"]}/100 - Skill Gap Analysis',
                 fontsize=15, fontweight='bold', pad=20)
    ax1.grid(axis='y', linestyle='--', alpha=0.3)
    ax1.set_xticklabels(skills, rotation=45, ha='right', fontsize=10)
    
    # Score breakdown
    score_labels = ['Skill Match\n(45%)', 'Experience\n(25%)', 'Projects\n(20%)', 'Bonus\n(10%)']
    score_values = [
        report['scores']['skill_match'],
        report['scores']['experience_fit'],
        report['scores']['project_relevance'],
        report['scores']['bonus_signals']
    ]
    score_colors = ['#3498db', '#9b59b6', '#1abc9c', '#f39c12']
    
    bars2 = ax2.barh(score_labels, score_values, color=score_colors, 
                     edgecolor='black', linewidth=0.8, alpha=0.85)
    
    # Add value labels
    for bar, value in zip(bars2, score_values):
        width = bar.get_width()
        ax2.text(width + 1, bar.get_y() + bar.get_height()/2,
                f'{value:.1f}%',
                ha='left', va='center', fontsize=10, fontweight='bold')
    
    ax2.set_xlim(0, 105)
    ax2.set_xlabel('Component Score (%)', fontsize=11, fontweight='bold')
    ax2.set_title('Score Components Breakdown', fontsize=12, fontweight='bold', loc='left')
    ax2.grid(axis='x', linestyle='--', alpha=0.3)
    
    # Legend
    legend_elements = [
        plt.Rectangle((0,0),1,1, fc="#2ecc71", ec="black", label="Required Skills (Matched)"),
        plt.Rectangle((0,0),1,1, fc="#3498db", ec="black", label="Preferred Skills (Matched)"),
        plt.Rectangle((0,0),1,1, fc="#f39c12", ec="black", label="Preferred Skills (Missing)"),
        plt.Rectangle((0,0),1,1, fc="#e74c3c", ec="black", label="Required Skills (Missing)")
    ]
    ax1.legend(handles=legend_elements, loc='upper right', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✅ Heatmap generated for {len(skills)} skills")

print("✅ Visualization functions loaded")

## Test the Model

Upload your resume and job description files to test the ATS scoring system.

In [ ]:
# Provide file paths for testing
# Replace with your actual file paths

RESUME_PATH = "path/to/resume.pdf"  # or .docx, .txt
JD_PATH = "path/to/job_description.pdf"  # or .docx, .txt

# For testing, you can create sample text
sample_resume = """
John Doe
Software Engineer
Email: john@email.com | Phone: 1234567890

EXPERIENCE
Senior Software Developer at TechCorp (2020-Present)
- 4 years of experience in full-stack development
- Led development of microservices using Python and Django
- Worked with React, Node.js, and PostgreSQL databases
- Implemented CI/CD pipelines using Docker and Jenkins

PROJECTS
E-Commerce Platform: Built scalable REST API using Python Flask and MongoDB
Real-time Chat Application: Developed using React and WebSockets
Data Analytics Dashboard: Created with Python, Pandas, and Matplotlib

SKILLS
Python, JavaScript, React, Node.js, SQL, PostgreSQL, MongoDB, Docker, Git, 
HTML, CSS, REST API, Agile, Unit Testing

EDUCATION
Bachelor of Technology in Computer Science (2020)

CERTIFICATIONS
AWS Certified Developer - Associate
Google Cloud Professional Certificate
"""

sample_jd = """
Full Stack Developer - TechStartup Inc.

REQUIREMENTS:
We are looking for an experienced Full Stack Developer with 3+ years of experience.

Required Skills:
- Python (Django/Flask)
- JavaScript (React or Vue)
- SQL databases (PostgreSQL/MySQL)
- RESTful API development
- Git version control
- HTML/CSS

Preferred Skills:
- Docker and containerization
- AWS or cloud platforms
- GraphQL
- Kubernetes

Minimum Experience: 3 years of professional software development

About the Role:
Mid-level position working on cutting-edge web applications. You'll collaborate
with a team to build scalable, high-performance systems.
"""

print("✅ Sample data ready")
print("\n💡 TIP: Replace RESUME_PATH and JD_PATH with actual file paths,")
print("   or use the sample_resume and sample_jd variables for testing")

In [ ]:
# Initialize the ATS Matcher
matcher = ATSMatcher(SKILL_DATABASE, SCORING_WEIGHTS)

# Extract text from files or use sample data
try:
    # Try loading from files
    resume_text = extract_text(RESUME_PATH)
    jd_text = extract_text(JD_PATH)
    print("✅ Loaded from files")
except:
    # Use sample data
    resume_text = sample_resume
    jd_text = sample_jd
    print("✅ Using sample data")

# Perform matching
print("\n🔍 Analyzing resume against job description...\n")
report = matcher.match(resume_text, jd_text)

# Print detailed report
matcher.print_detailed_report(report)

In [ ]:
# Generate visual heatmap
plot_enhanced_heatmap(report)

## Recommendations

Based on the ATS score, here are personalized recommendations for improvement.

In [ ]:
def generate_recommendations(report):
    """Generate personalized recommendations based on score"""
    
    print("\n" + "="*70)
    print("               PERSONALIZED RECOMMENDATIONS")
    print("="*70)
    
    score = report['final_score']
    
    # Priority actions based on score
    if score >= 85:
        print("\n🎉 EXCELLENT MATCH!")
        print("   Your profile strongly aligns with this role.")
        print("\n   Action Items:")
        print("   1. Apply immediately - you're a strong candidate")
        print("   2. Tailor your cover letter to highlight matching skills")
        print("   3. Prepare to discuss your relevant projects in detail")
        
    elif score >= 70:
        print("\n✅ STRONG CANDIDATE")
        print("   You meet most requirements for this role.")
        print("\n   Action Items:")
        print("   1. Apply with confidence")
        print("   2. Address any missing skills in your cover letter")
        print("   3. Highlight your transferable experience")
        
    elif score >= 55:
        print("\n📚 DEVELOPING CANDIDATE")
        print("   You have good foundation but some skill gaps.")
        print("\n   Action Items:")
        print("   1. Consider applying if you're a fast learner")
        print("   2. Take online courses for missing critical skills")
        print("   3. Build projects showcasing required technologies")
        
    else:
        print("\n🎯 EARLY STAGE")
        print("   Significant gap between your profile and this role.")
        print("\n   Action Items:")
        print("   1. Focus on building required skills first")
        print("   2. Look for junior/entry-level positions")
        print("   3. Create portfolio projects using required technologies")
    
    # Skill-specific recommendations
    if report['missing_required']:
        print(f"\n🎯 CRITICAL SKILLS TO ACQUIRE ({len(report['missing_required'])}):")
        for i, skill in enumerate(sorted(report['missing_required'])[:5], 1):
            print(f"   {i}. {skill.upper()}")
            
            # Suggest resources
            if skill == 'python':
                print("      → Learn: Python.org tutorial, Automate the Boring Stuff")
            elif skill == 'react':
                print("      → Learn: React official docs, FreeCodeCamp React course")
            elif skill == 'docker':
                print("      → Learn: Docker official docs, Docker Mastery course")
            elif skill == 'sql':
                print("      → Learn: SQLBolt, W3Schools SQL tutorial")
    
    # Bonus opportunities
    if report['preferred_missing'] and score >= 60:
        print(f"\n⭐ BONUS LEARNING OPPORTUNITIES ({len(report['preferred_missing'])}):")
        print("   These will make you stand out:")
        for i, skill in enumerate(sorted(report['preferred_missing'])[:3], 1):
            print(f"   {i}. {skill.upper()}")
    
    # Experience recommendations
    resume_exp = report['resume_details']['experience_years']
    required_exp = report['jd_details']['required_experience']
    
    if resume_exp < required_exp:
        gap = required_exp - resume_exp
        print(f"\n📈 EXPERIENCE GAP: Need {gap:.1f} more years")
        print("   → Emphasize relevant project work and internships")
        print("   → Highlight impact and responsibilities in current role")
    
    print("\n" + "="*70)

# Generate recommendations
generate_recommendations(report)